# Coupled higgs equation 
$$
u_{tt} - u_{xx} + |u|^2 u - 2uv = 0
$$
$$
v_{tt} + v_{xx} - (\left| u \right|^2)_{xx} = 0
$$

where, $ u(x,t) $ represents a complex nucleon field and $ v(x,t) $ represents a real scalar meson field. The coupled Higgs field Equation describes a system of conserved scalar nucleon interaction with a neutral scalar meson.

solutions 

$$
u_1(x, t) = ir e^{ir(\omega x + t)} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$
$$
v_1(x, t) = r^2 \tanh^2\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$

for $t = 0$

$$
u_1(x, 0) = ir e^{ir \omega x} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$
$$
v_1(x, 0) = r^2 \tanh^2\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$

where 
$k = 4, \omega = 5 , \alpha = 2, c = 2, r = 2$


In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.init as init
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)
#device = torch.device('cpu')

In [2]:
def fourier_features(x, B):
    x_transformed = torch.matmul(x, B)
    return torch.cat([torch.sin(2 * torch.pi * x_transformed), torch.cos(2 * torch.pi * x_transformed)], dim=-1)

def init_frequency_matrix(size, std_dev=1.0):
    B = torch.normal(0, std_dev, size=size)
    return B

class FourierFeatureNN(nn.Module):
    def __init__(self, input_dim=1, shared_units=128, output_dim=3, layers_per_path=2, std_dev=1.0, activation=nn.Tanh, device=device):
        super(FourierFeatureNN, self).__init__()
        self.Bx = init_frequency_matrix((input_dim, shared_units // 2), std_dev=std_dev).float().to(device)
        self.Bt = init_frequency_matrix((input_dim, shared_units // 2), std_dev=std_dev).float().to(device)

        # Separate paths for processing Fourier features of x and t with weighting
        self.path_x_weights = nn.Sequential(
            nn.Linear(shared_units, 256),
            activation(),
            nn.Linear(256, 128),
            activation(),
            nn.Linear(128, 64),
            activation()
        )

        for layer in self.path_x_weights:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)

        # And a similar structure for t, but with different sizes as an example
        self.path_t_weights = nn.Sequential(
            nn.Linear(shared_units, 256),
            activation(),
            nn.Linear(256, 128),
            activation(),
            nn.Linear(128, 64),
            activation()
        )

        # Repeat weight initialization for path_t_weights
        for layer in self.path_t_weights:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)
                    
        self.combine_and_output = nn.Sequential(
            nn.Linear(64, 32),  # Assuming we're combining the last layers of path_x and path_t
            activation(),
            nn.Linear(32, 16),
            activation(),
            nn.Linear(16, output_dim)  # Output dimension remains 3 (u_real, u_imag, v)
        )

        for layer in self.combine_and_output:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)

    def forward(self, x, t):
        # Transform x and t into Fourier features
        x_fourier = fourier_features(x, self.Bx)
        t_fourier = fourier_features(t, self.Bt)

        # Get weighted Fourier features for x and t
        weighted_x = self.path_x_weights(x_fourier)
        weighted_t = self.path_t_weights(t_fourier)

        # Element-wise multiplication of weighted Fourier features of x and t
        combined_features = weighted_x * weighted_t

        # Produce u_real, u_imag, and v by passing combined features through final layer
        output = self.combine_and_output(combined_features)
        u_real, u_imag, v = output[:, 0], output[:, 1], output[:, 2]

        return u_real.view(-1, 1), u_imag.view(-1, 1), v.view(-1, 1)


In [3]:
def grad(y, x):
    return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True, retain_graph=True)[0]

def laplacian(field, x, t):
    field_x = grad(field, x)
    field_xx = grad(field_x, x)
    field_t = grad(field, t)
    field_tt = grad(field_t, t)
    return field_xx, field_tt

# Define the ODE system for the Coupled Higgs field equations
def coupled_higgs(u_real, u_imag, v, x, t):
    u_r_xx, u_r_tt = laplacian(u_real, x, t)
    u_i_xx, u_i_tt = laplacian(u_imag, x, t)
    v_xx, v_tt = laplacian(v, x, t)

    u_abs = u_real**2 + u_imag**2
    u_abs_xx, u_abs_tt = laplacian(u_abs, x, t)

    # Calculate the field equations
    du_eq_r = u_r_tt - u_r_xx + u_abs * u_real - 2 * u_real * v
    du_eq_i = u_i_tt - u_i_xx + u_abs * u_imag - 2 * u_imag * v
    dv_eq = v_tt + v_xx - u_abs_xx
    
    return du_eq_r, du_eq_i, dv_eq

# Function to calculate the real part of u1
def real_u1(x, t, k, omega, r):
    return np.real(1j * r * np.exp(1j * r * (omega * x + t)) * np.sqrt(1 + omega**2) *
                   np.tanh( (r * (k + x + omega * t)) / np.sqrt(2) ) )

def imag_u1(x, t, k, omega, r):
    return np.imag(1j * r * np.exp(1j * r * (omega * x + t)) * np.sqrt(1 + omega**2) *
                   np.tanh((r * (k + x + omega * t)) / np.sqrt(2) ) )
    
def real_v1(x, t, k, omega, r):
    return (r * np.tanh((r * (k + x + omega * t)) / np.sqrt(2)) )**2

def compute_analytical_boundary_loss(model, x_boundary, t_boundary, mse_cost_function, k, omega, r):
    x = torch.from_numpy(x_boundary).float().to(device)
    t = torch.from_numpy(t_boundary).float().to(device)
    pred_u_r, pred_u_i, pred_v = model(x, t)

    real_u1_val = torch.tensor(real_u1(x_boundary, t_boundary, k, omega, r), device=device).float().view(-1, 1)
    imag_u1_val = torch.tensor(imag_u1(x_boundary, t_boundary, k, omega, r), device=device).float().view(-1, 1)
    real_v1_val = torch.tensor(real_v1(x_boundary, t_boundary, k, omega, r), device=device).float().view(-1, 1)
 
    boundary_loss_ur = mse_cost_function(pred_u_r, real_u1_val)
    boundary_loss_ui = mse_cost_function(pred_u_i, imag_u1_val)
    boundary_loss_v = mse_cost_function(pred_v, real_v1_val)
    
    return boundary_loss_ur, boundary_loss_ui, boundary_loss_v

def compute_physics_loss(model, x, t, mse_cost_function):
    pred_u_r, pred_u_i, pred_v = model(x, t)

    # Compute the differential equation residuals
    du_eq_r, du_eq_i, dv_eq = coupled_higgs(pred_u_r, pred_u_i, pred_v, x, t)
    
    # Define target tensors of zeros with the same shape as the predictions
    zeros_r = torch.zeros_like(du_eq_r, device=device)
    zeros_i = torch.zeros_like(du_eq_i, device=device)
    zeros_v = torch.zeros_like(dv_eq, device=device)
    
    # Compute the MSE loss against zeros for each differential equation residual
    loss_r = mse_cost_function(du_eq_r, zeros_r)
    loss_i = mse_cost_function(du_eq_i, zeros_i)
    loss_v = mse_cost_function(dv_eq, zeros_v)
    
    # Return the scalar loss values for real part, imaginary part, and v
    return loss_r, loss_i, loss_v

In [ ]:
# Check if CUDA is available and set the default device
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

model = FourierFeatureNN(input_dim=1, shared_units=128, output_dim=3, layers_per_path=3, std_dev=1.0, activation=nn.Tanh, device=device).to(device)


num_epochs = 100000  # Number of training epochs
lr = 1e-3          # Learning rate
num_samples = 1000 # Number of samples for training
r = 1.1
omega = 5
k = 0.5

optimizer = Adam(model.parameters(), lr=lr)
mse_cost_function = torch.nn.MSELoss()
model_save_path = 'model_weights_testing_CHIGGS_fourier_collective'
os.makedirs(model_save_path, exist_ok=True)
losses = []

# Training loop
for epoch in tqdm(range(num_epochs),
                  desc='Progress:',  # Empty description
                  leave=False,  # Do not leave the progress bar when done
                  ncols=75,  # Width of the progress bar
                  mininterval=0.1,
                  bar_format='{l_bar}{bar}|{remaining}',  # Only show the bar without any counters
                  colour='blue'):
    x_n = (torch.rand(num_samples, 1) * 1).to(device)  # x in range [-5, -3]
    t_n = (torch.rand(num_samples, 1) * 1).to(device)   
    x_n.requires_grad = True
    t_n.requires_grad = True
    x_bc_x0 = np.zeros((num_samples, 1))
    t_bc_x0 = np.random.uniform(0, 1, (num_samples, 1))
    x_bc_x1 = np.ones((num_samples, 1))
    t_bc_x1 = np.random.uniform(0, 1, (num_samples, 1))
    x_bc_t0 = np.random.uniform(0, 1, (num_samples, 1))
    t_bc_t0 = np.zeros((num_samples, 1))
    x_bc_t1 = np.random.uniform(0, 1, (num_samples, 1))
    t_bc_t1 = np.ones((num_samples, 1))
    x_dom = np.random.uniform(0, 1, (num_samples, 1))
    t_dom = np.random.uniform(0, 1, (num_samples, 1))
    
    optimizer.zero_grad()

    physics_loss_ur, physics_loss_ui, physics_loss_v = compute_physics_loss(model, x_n, t_n, mse_cost_function)
    boundary_loss_ur_x0, boundary_loss_ui_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model, x_bc_x0, t_bc_x0, mse_cost_function, k, omega, r)
    boundary_loss_ur_x1, boundary_loss_ui_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model, x_bc_x1, t_bc_x1, mse_cost_function, k, omega, r)
    boundary_loss_ur_t0, boundary_loss_ui_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model, x_bc_t0, t_bc_t0, mse_cost_function, k, omega, r)
    boundary_loss_ur_t1, boundary_loss_ui_t1, boundary_loss_v_t1 = compute_analytical_boundary_loss(model, x_bc_t1, t_bc_t1, mse_cost_function, k, omega, r)

    domain_loss_ur_t, domain_loss_ui_t, domain_loss_v_t = compute_analytical_boundary_loss(model, x_dom, t_dom, mse_cost_function, k, omega, r)
   
    # Total loss 
    loss_ur = physics_loss_ur + boundary_loss_ur_x0 + boundary_loss_ur_x1 + boundary_loss_ur_t0 + boundary_loss_ur_t1 + domain_loss_ur_t
    loss_ui = physics_loss_ui + boundary_loss_ui_x0 + boundary_loss_ui_x1 + boundary_loss_ui_t0 + boundary_loss_ui_t1 + domain_loss_ui_t
    loss_v = physics_loss_v + boundary_loss_v_x0 + boundary_loss_v_x1 + boundary_loss_v_t0 + boundary_loss_v_t1 + domain_loss_v_t

    total_loss = loss_ur + loss_ui + loss_v
    
    total_loss.backward()
    optimizer.step()
    
    # Print loss every few epochs
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss U (real): {loss_ur.item()}, Loss U (imag): {loss_ui.item()}, Loss V: {loss_v.item()}')
        model_filename = os.path.join(model_save_path, f'C_HIGGS_collective_epoch_{epoch}.pth')
        torch.save(model.state_dict(), model_filename)
        
        df_losses = pd.DataFrame(losses)
        csv_file_path = 'loss_data/C_HIGGS_fourier_training_collective_losses.csv'
        df_losses.to_csv(csv_file_path, index=False)
    
    if total_loss.item() < 0.01:
        print(f'Stopping early at epoch {epoch} due to reaching target loss.')
        break
    
    losses.append({
        'Epoch': epoch,
        'Loss U (Real)': loss_ur.item(),
        'Loss U (Imag)': loss_ui.item(),
        'Loss V': loss_v.item(),
        'Total Loss': total_loss.item(),
        'Physics Loss': physics_loss_ur.item() + physics_loss_ui.item() + physics_loss_v.item(),
        'Boundary Loss U(Real)': boundary_loss_ur_x0.item() + boundary_loss_ur_x1.item() + boundary_loss_ur_t0.item() + boundary_loss_ur_t1.item(),
        'Boundary Loss U(Imag)': boundary_loss_ui_x0.item() + boundary_loss_ui_x1.item() + boundary_loss_ui_t0.item() + boundary_loss_ui_t1.item(),
        'Boundary Loss V': boundary_loss_v_x0.item() + boundary_loss_v_x1.item() + boundary_loss_v_t0.item() + boundary_loss_v_t1.item(),
        'Domain Loss': domain_loss_ui_t + domain_loss_ur_t + domain_loss_v_t
    })

CUDA is available! Training on GPU.


Progress::   0%|                                                  |21:20:06

Epoch 0, Loss U (real): 1160.03173828125, Loss U (imag): 3703.409912109375, Loss V: 7313.49560546875


Progress::   1%|▌                                                 |11:16:39

Epoch 1000, Loss U (real): 31.595775604248047, Loss U (imag): 58.359954833984375, Loss V: 2.662620782852173


...

Progress::  11%|█████▌                                            |14:52:38

Epoch 11000, Loss U (real): 23.47681999206543, Loss U (imag): 51.096717834472656, Loss V: 5.296541690826416


Progress::  12%|██████                                            |12:34:48

Epoch 12000, Loss U (real): 23.737773895263672, Loss U (imag): 50.93486022949219, Loss V: 4.377358436584473


Progress::  12%|██████                                            |11:41:50

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate a grid of x and t values
x = np.linspace(0, 1, 400)  # x values
t = np.linspace(0, 1, 400)  # t values
X, T = np.meshgrid(x, t)
X_flat = X.reshape(-1, 1)  # Reshape to (160000, 1) instead of flattening to (160000,)
T_flat = T.reshape(-1, 1)  # Reshape to (160000, 1)

# Convert to torch tensors and prepare for the model
X_tensor = torch.from_numpy(X_flat).float().to(device)
T_tensor = torch.from_numpy(T_flat).float().to(device)

# Convert to torch tensors and prepare for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming shared_model is defined and loaded with trained parameters
model_state = torch.load(os.path.join(model_save_path, 'C_HIGGS_collective_epoch_99000.pth'), map_location=device)

model.load_state_dict(model_state)
model.eval()

# Get predictions from the trained models
with torch.no_grad():
    pred_u_r, pred_u_i, pred_v = model(X_tensor, T_tensor)

    # Reshape predictions to match the grid shape for plotting
    pred_u_r = pred_u_r.cpu().numpy().reshape(X.shape)
    pred_u_i = pred_u_i.cpu().numpy().reshape(X.shape)
    pred_v = pred_v.cpu().numpy().reshape(X.shape)

# Calculate the analytical solutions
real_u1_analytical = real_u1(X, T, k, omega, r)
imag_u1_analytical = imag_u1(X, T, k, omega, r)
real_v1_analytical = real_v1(X, T, k, omega, r)

# Plotting
fig = plt.figure(figsize=(20, 6))

# Plot predicted and analytical real part of u1
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X, T, pred_u_r, cmap='viridis')
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
ax1.set_title('Predicted Real Part of $u_1(x, t)$')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('Real part of $u_1$')

# Plot predicted and analytical imaginary part of u1
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X, T, pred_u_i, cmap='viridis')
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=5)
ax2.set_title('Predicted Imaginary Part of $u_1(x, t)$')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('Imag part of $u_1$')

# Plot predicted and analytical real part of v1
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(X, T, pred_v, cmap='viridis')
fig.colorbar(surf3, ax=ax3, shrink=0.5, aspect=5)
ax3.set_title('Predicted Real Part of $v_1(x, t)$')
ax3.set_xlabel('x')
ax3.set_ylabel('t')
ax3.set_zlabel('Real part of $v_1$')

plt.tight_layout()
plt.show()

# Plotting
fig = plt.figure(figsize=(20, 6))

# Plot predicted and analytical real part of u1
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X, T, real_u1_analytical, cmap='viridis')
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
ax1.set_title('Analytical Real Part of $u_1(x, t)$')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('Real part of $u_1$')

# Plot predicted and analytical imaginary part of u1
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X, T, imag_u1_analytical, cmap='viridis')
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=5)
ax2.set_title('Analytical Imaginary Part of $u_1(x, t)$')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('Imag part of $u_1$')

# Plot predicted and analytical real part of v1
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(X, T, real_v1_analytical, cmap='viridis')
fig.colorbar(surf3, ax=ax3, shrink=0.5, aspect=5)
ax3.set_title('Analytical Real Part of $v_1(x, t)$')
ax3.set_xlabel('x')
ax3.set_ylabel('t')
ax3.set_zlabel('Real part of $v_1$')

plt.tight_layout()
plt.show()

plt.close()